### Learning Sockets in Python

This notebook is about programming with Sockets in Python. This is required for practicing the Pyspark streaming, and creating data streams from various IOT devices and services. 

The socket API for Internet sockets, sometimes called Berkeley or BSD sockets. There are also Unix domain sockets, which can only be used to communicate between processes on the same host.


The primary socket API functions and methods in this module are:

.socket()

.bind()

.listen()

.accept()

.connect()

.connect_ex()

.send()

.recv()

.close()

In [5]:
import socket

HOST = "127.0.0.1"
PORT = 65430

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as tcp:
    tcp.bind((HOST, PORT))
    tcp.listen()
    cont, adrs = tcp.accept()
    #The .accept() method blocks execution and waits for an incoming connection.
    with cont:
        print(f"Connected with {adrs}")
        while True:
            data = cont.recv(1024)
            print(data)
            if not data:
                break
            cont.sendall(data)

Connected with ('127.0.0.1', 40450)
b'Hello, worldWorld has become brave new now'
b''


In [ ]:
import socket

HOST = "127.0.0.1"
PORT = 65430

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((HOST, PORT))
    #The above connect is temporary. Once program ends the connection stops
    s.sendall(b"Hello, world")
    data = s.recv(1024)
    
print(f"Recieved {data}")

In [6]:
#Lets build a non-blocking sockets
import sys
import selectors
import types

In [16]:
def accept_wrapper(sock):
    conn, addr = sock.accept()  # Should be ready to read
    print(f"Accepted connection from {addr}")
    conn.setblocking(False)
    data = types.SimpleNamespace(addr=addr, inb=b"", outb=b"")
    events = selectors.EVENT_READ | selectors.EVENT_WRITE
    sel.register(conn, events, data=data)

In [17]:
def service_connection(key, mask):
    sock = key.fileobj
    data = key.data
    if mask & selectors.EVENT_READ:
        recv_data = sock.recv(1024)  # Should be ready to read
        if recv_data:
            data.outb += recv_data
        else:
            print(f"Closing connection to {data.addr}")
            sel.unregister(sock)
            sock.close()
    if mask & selectors.EVENT_WRITE:
        if data.outb:
            print(f"Echoing {data.outb!r} to {data.addr}")
            sent = sock.send(data.outb)  # Should be ready to write
            data.outb = data.outb[sent:]

In [19]:
sel = selectors.DefaultSelector()


host, port = '127.0.0.1',65280
lsock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
lsock.bind((host,port))
lsock.listen()
print(f"Listening on {(host, port)}")
lsock.setblocking(False)
sel.register(lsock,selectors.EVENT_READ, data=None)

try:
    while True:
        events = sel.select(timeout=None)
        for key, mask in events:
            if key.data is None:
                accept_wrapper(key.fileobj)
            else:
                service_connection(key,mask)
except KeyboardInterrupt:
    print("caught Key interrupt")

finally:
    sel.close()

Listening on ('127.0.0.1', 65280)
Accepted connection from ('127.0.0.1', 37630)
Echoing b'Hello, worldWorld has become brave new now' to ('127.0.0.1', 37630)
Closing connection to ('127.0.0.1', 37630)
Accepted connection from ('127.0.0.1', 54938)
Echoing b'Hello, worldWorld has become brave new now' to ('127.0.0.1', 54938)
Closing connection to ('127.0.0.1', 54938)
Accepted connection from ('127.0.0.1', 54940)
Echoing b'Hello, worldWorld has become brave new now' to ('127.0.0.1', 54940)
Closing connection to ('127.0.0.1', 54940)
caught Key interrupt


In [21]:
sel = selectors.DefaultSelector()
messages = [b"Message 1 from client.", b"Message 2 from client."]

def start_connections(host, port, num_conns):
    server_addr = (host, port)
    for i in range(0, num_conns):
        connid = i + 1
        print(f"Starting connection {connid} to {server_addr}")
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.setblocking(False)
        sock.connect_ex(server_addr)
        events = selectors.EVENT_READ | selectors.EVENT_WRITE
        data = types.SimpleNamespace(
            connid=connid,
            msg_total=sum(len(m) for m in messages),
            recv_total=0,
            messages=messages.copy(),
            outb=b"",
        )
        sel.register(sock, events, data=data)

In [34]:
 def service_connection(key, mask):
    sock = key.fileobj
    data = key.data
    if mask & selectors.EVENT_READ:
        recv_data = sock.recv(1024)  # Should be ready to read
        if recv_data:
          data.outb += recv_data
          print(f"Received {recv_data!r} from connection {data.connid}")
          data.recv_total += len(recv_data)
        else:
          print(f"Closing connection {data.connid}")
        if not recv_data or data.recv_total == data.msg_total:
          print(f"Closing connection {data.connid}")
          sel.unregister(sock)
          sock.close()
    if mask & selectors.EVENT_WRITE:
        if not data.outb and data.messages:
            data.outb = data.messages.pop(0)
        if data.outb:
            print(f"Echoing {data.outb!r} to {data.addr}")
            print(f"Sending {data.outb!r} to connection {data.connid}")
            sent = sock.send(data.outb)  # Should be ready to write
            data.outb = data.outb[sent:]

What you’re going to do is move the message code into a class named Message and add methods to support reading, writing, and processing of the headers and content. This is a great example for using a class.